In [1]:
%cd ..

/tf/notebooks


In [2]:
!pip install ijson

You are using pip version 18.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
import ijson
from tqdm import tqdm_notebook as tqdm

In [4]:
import json

In [5]:
ingredients = []
with open("data/recipes/recipes_matched.jsonl", encoding='utf-8') as f:
    for line in tqdm(f):
        recipe = json.loads(line)
        ingredients += [ingredient for ingredient in recipe["ingredients"] if ingredient["type"] == "ingredient" and ingredient["matched"]["matched"]]

In [6]:
from collections import defaultdict, Counter
orig_frequencies = defaultdict(Counter)
frequencies = Counter()
ings = {}
for ingredient in tqdm(ingredients):
    orig_text = ingredient["original"]["ingredient"]
    mid = ingredient["matched"]["id"]
    frequencies.update([mid])
    ings[mid] = ingredient["matched"]
    orig_frequencies[mid].update([orig_text])

In [11]:
for id, count in frequencies.most_common(100):
    origs = orig_frequencies[id].most_common(5)
    print((str(count)+ "x").rjust(10), "", ings[id]["name"], [o for o, count in origs])

   119244x  Salz ['Salz', 'Salz und Pfeffer', 'Salz und Pfeffer aus der Mühle', 'Salz und Pfeffer, aus der Mühle', 'Salz und Pfeffer, weißer']
    59066x  Zucker ['Zucker', 'Zucker, braun', 'Zucker, brauner', 'Zucker, weiß', 'Zucker, feiner']
    58185x  Ei, von der Gans / Gänseei ['Ei(er)', 'Ei(er), getrennt', 'Ei(er), hart gekocht', 'Ei(er), Größe M', 'Ei(er), davon das Eigelb']
    46069x  Mehl ['Mehl', 'Mehl (Weizenmehl)', 'Mehl, gesiebt', 'Mehl, Type 405', 'Mehl, glattes']
    45891x  Butter, streichfein, gesalzen ['Butter', 'Butter, weiche', 'Butter, weich', 'Butter oder Margarine', 'Butter, zerlassen']
    41206x  Rote Zwiebel, frisch ['Zwiebel(n)', 'Zwiebel(n), rot', 'Zwiebel(n), gewürfelt', 'Zwiebel(n), gehackt', 'Zwiebel(n), fein gehackt']
    24531x  Milch, laktosefrei (3,8 %) ['Milch', 'Milch, lauwarm', 'Milch, lauwarme', 'Milch, fettarme', 'Milch, fettarm']
    24011x  Vanillezucker ['Vanillezucker', 'Vanillinzucker', 'Vanillezucker, Bourbon-', 'Vanillezucker (Bourbon)', '

In [9]:
ing_freqs = []
for id, count in frequencies.most_common():
    if count < 5:
        continue
    origs = orig_frequencies[id].most_common(5)
    common_source_names = [o for o, count in origs]
    ing_freqs.append({**ings[id], "most_common_matches": common_source_names})

with open("data/recipes/ingredients_common.json", "w") as f:
    json.dump(ing_freqs, f)